In [ ]:
!pip install langchain
!pip install openai
!pip install sentence-transformers
!pip install faiss-cpu
!pip install langchainhub
!pip install tiktoken
!pip install langchain_openai
!pip install --upgrade rank_bm25
!pip install langchain-community langchain-core
!pip install -U langchain langchain-openai
import os
from langchain.llms import OpenAI
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import TextLoader
from langchain.vectorstores import Chroma
from langchain.agents.agent_toolkits import (
    create_vectorstore_agent,
    VectorStoreToolkit,
    VectorStoreInfo
)
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.agents import Tool
from langchain.agents import AgentType
from langchain.memory import ConversationBufferMemory
from langchain.llms import OpenAI
from langchain.utilities import SerpAPIWrapper
from langchain.agents import initialize_agent
from langchain.chains import ConversationChain
from langchain.vectorstores import LanceDB
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.agents import AgentExecutor
from langchain_core.messages import AIMessage, HumanMessage
from langchain.agents import create_openai_functions_agent
from langchain import hub
from langchain_openai import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
from langchain.agents import create_openai_functions_agent
from langchain.agents import AgentExecutor
from langchain.tools.retriever import create_retriever_tool
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
import re
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor
from langchain.retrievers.multi_query import MultiQueryRetriever


In [ ]:
import os

os.environ['OPENAI_API_KEY'] = ''


import openai


os.environ['OPENAI_API_BASE'] = 'https://api.openai.com/v1'

embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

vector = FAISS.load_local("faiss_index", embedding_function,allow_dangerous_deserialization=True)

retriever = vector.as_retriever(search_kwargs={"k": 30})

llm = ChatOpenAI(model="gpt-4-turbo", temperature=0.01)

prompt = ChatPromptTemplate.from_messages(
    [
        (   "system",
            '''
            You are recognized as a leading authority in the field of solid state battery, tasked with the pivotal role of providing expert responses to inquiries.
            You are also good at visualizing data and drawing charts using Python libraries, for example, matplotlib, seaborn, or plotly. You can use these libraries to generate Python code that can visualize data and hence help explain your thoughts. When data visualization is necessary, generating Python code is enough. Do not try to execute the generated Python code.
            Your responses must embody the highest standards of accuracy, comprehensiveness, and depth of knowledge. Approach each question with a methodical and thoughtful mindset, ensuring your answers are:

            Informative: Furnish detailed insights, drawing upon the most relevant facts and figures.
            Correct: Ensure factual accuracy in every aspect of your response.
            Knowledgeable: Display a profound understanding of the subject matter, including advanced concepts and recent advancements in the field of solid state battery.
            Holistic: Offer a well-rounded perspective, considering various facets of the topic at hand.
            As you address each question, please adhere to the following guidelines:

            Cite Examples: When referencing data or examples from the provided literature, include comprehensive information to contextualize your points effectively. Clearly indicate these instances by stating, "For example," followed by a detailed explanation.
            Stay On Topic: Concentrate solely on the query posed. Your reply should be closely aligned with the question, avoiding tangential or unrelated content.
            Format Your Answer: Present your response in a structured manner, using either bullet points or numbered lists for clarity and ease of understanding.
            Before responding, take a moment to center yourself. Breathe deeply, and proceed with a step-by-step analytical approach, ensuring that your expertise shines through in a manner that is both engaging and enlightening.

            ''',
        ),
        MessagesPlaceholder(variable_name="chat_history"),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

retriever_tool = create_retriever_tool(
    retriever,
    "literature_search",
    "Search for information about input questions. For any questions about solid state battery, you must use this tool!",
)

tools = [retriever_tool]

agent = create_openai_functions_agent(llm, tools, prompt)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, reduce_k_below_max_tokens=True)


breakdown_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", '''
        You are an expert in solid state battery, and your task is to answer user's question professionally.
        You need to design several intermediate steps to enhance processing and reasoning, aiming to reach the final answers through the intermediate steps.
        Ensure that each intermediate step is interconnected and collectively addresses the user's inquiry. These intermediate thoughts should encompass various aspects of solid state battery, requiring information retrieval from a comprehensive library.
        Your approach should systematically break down the query, ensuring a thorough exploration of the topic from multiple angles to provide a well-rounded answer.
        Your steps needs to be closely involve different aspects in solid state battery and material science, you must contain domain knowledge aspects in each step, and include reasoning thoughts
        You only need to list the intermeditae steps.
        Your output should be in unified format, for each step, encapsulate it as
        [step]
        the content for step1
        [step]
        the content for step2
        [step]
        the content for step3
        ......
        '''),
        MessagesPlaceholder(variable_name="chat_history"),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)


agent_breakdown = create_openai_functions_agent(llm, tools, breakdown_prompt)

agent_breakdown_executor = AgentExecutor(agent=agent_breakdown, tools=tools, verbose=True, reduce_k_below_max_tokens=True)


breakdown_a=agent_breakdown_executor.invoke({"input":input, 'chat_history':[]})

thoughts=breakdown_a['output']

step_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", '''
        You are an expert in solid state battery, and your task is to answer user's question professionally.
       You must answer questions about solid-state batteries in only one step
        Informative: Furnish detailed insights, drawing upon the most relevant facts and figures.
        Correct: Ensure factual accuracy in every aspect of your response.
        Knowledgeable: Display a profound understanding of the subject matter, including advanced concepts and recent advancements in the field of solid state battery.
        Holistic: Offer a well-rounded perspective, considering various facets of the topic at hand.
        As you address each question, please adhere to the following guidelines:

        Stay On Topic: Concentrate solely on the query posed. Your reply should be closely aligned with the question, avoiding tangential or unrelated content.When you write drawing code, try your best to make the pictures drawn by your code clear and beautiful.
        Before responding, take a moment to center yourself. Breathe deeply, and proceed with a step-by-step analytical approach, ensuring that your expertise shines through in a manner that is both engaging and enlightening.
        You are also good at visualizing data and drawing charts using Python libraries, for example, matplotlib, seaborn, or plotly. You can use these libraries to generate Python code that can visualize data and hence help explain your thoughts. When data visualization is necessary, generating Python code is enough. Do not try to execute the generated Python code.
        '''),
        MessagesPlaceholder(variable_name="chat_history"),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

agent_step = create_openai_functions_agent(llm, tools, step_prompt)

agent_step_executor = AgentExecutor(agent=agent_step, tools=tools, verbose=True, reduce_k_below_max_tokens=True)

input= ""

breakdown_a=agent_breakdown_executor.invoke({"input":input, 'chat_history':[]})

thoughts=breakdown_a['output']
thought=agent_step_executor.invoke({"input": "To address the question "+input+", we will first analyze a series of intermediate thoughts laid out for you. "+thoughts, 'chat_history':[]})
thoughts+=thought['output']
show_response(thoughts)



In [ ]:
import re
def show_response(answer):
    if isinstance(answer, dict) and 'output' in answer:
        response = answer['output']
    elif isinstance(answer, str):
        response = answer
    else:
        print(f"Invalid response type: {type(answer)}")
        return
    print(f"The response is: {response}")
    x = re.search(r'```python\n(.*?)\n```', response, re.DOTALL)
    if x is not None:
        code = x.group(1)
        exec(code)